In [6]:
!gdown 1sO1vBlx0-pPwvNr5BycSE-Mhqq2Xc3Lx

!gdown 15J0ioY2n0GY-iD1BzR5r62o7JpTtU_Mb

!gdown 1-1L2_l6q01aD7Zsw_XMeVQfKtwP031Y9

Downloading...
From (original): https://drive.google.com/uc?id=1sO1vBlx0-pPwvNr5BycSE-Mhqq2Xc3Lx
From (redirected): https://drive.google.com/uc?id=1sO1vBlx0-pPwvNr5BycSE-Mhqq2Xc3Lx&confirm=t&uuid=94d15035-1d71-4bf1-957d-54bc3397b3f4
To: /content/obligation_classification_model.pt
100% 268M/268M [00:01<00:00, 202MB/s]
Downloading...
From: https://drive.google.com/uc?id=15J0ioY2n0GY-iD1BzR5r62o7JpTtU_Mb
To: /content/test_obligation_data.csv
100% 45.4k/45.4k [00:00<00:00, 94.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1L2_l6q01aD7Zsw_XMeVQfKtwP031Y9
To: /content/extended_obligation_classification_dataset_V2.csv
100% 15.3M/15.3M [00:00<00:00, 73.7MB/s]


In [7]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# Load the Saved Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=2)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)
model.load_state_dict(torch.load('obligation_classification_model.pt', map_location=device))
model.to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define Parameters
max_len = 64  # Set the maximum length for tokenization

# Method to Classify a Given Sentence and Print the Result
def classify_sentence(model, sentence):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).cpu().numpy()[0]

    label_map = {1: 'obligation', 0: 'non-obligation'}
    print(f'Sentence: "{sentence}"')
    print(f'Classification: {label_map[prediction]}')
    print()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Test sentences
obligation_sentences = [
    "Ensure all financial reports are submitted by the end of the fiscal year.",
    "All vendors must submit their invoices within 30 days of service completion.",
    "Comply with the regulations set forth by the governing body.",
    "Employees are mandated to wear identification badges at all times within the office premises.",
    "Contractors must adhere to the safety guidelines outlined in the project manual.",
    "The organization shall ensure that customer data is encrypted during transmission and storage.",
    "Disclose all material facts to shareholders and investors.",
    "Meet the regulatory standards for financial reporting."
]

non_obligation_sentences = [
    "Entities found in violation of Section 92(6) may face substantial fines or other regulatory actions.",
    "Rules: Modification, Waiver, Contravention and Procedural Provisions ",
    "The team decided to have a meeting every Monday morning.",
    "Employees are encouraged to take regular breaks to maintain productivity.",
    "The company offers flexible working hours to its staff.",
    "The CEO announced that the annual picnic will be held in July.",
    "It is recommended that employees use secure passwords for their accounts.",
    "Company holiday parties are scheduled for December.",
    "The office will be closed for maintenance next week."
]

# Classify test sentences
for sentence in obligation_sentences:
    classify_sentence(model, sentence)

print("NON OBLIGATIONS")
for sentence in non_obligation_sentences:
    classify_sentence(model, sentence)

Sentence: "Ensure all financial reports are submitted by the end of the fiscal year."
Classification: obligation

Sentence: "All vendors must submit their invoices within 30 days of service completion."
Classification: obligation

Sentence: "Comply with the regulations set forth by the governing body."
Classification: obligation

Sentence: "Employees are mandated to wear identification badges at all times within the office premises."
Classification: obligation

Sentence: "Contractors must adhere to the safety guidelines outlined in the project manual."
Classification: obligation

Sentence: "The organization shall ensure that customer data is encrypted during transmission and storage."
Classification: obligation

Sentence: "Disclose all material facts to shareholders and investors."
Classification: obligation

Sentence: "Meet the regulatory standards for financial reporting."
Classification: non-obligation

NON OBLIGATIONS
Sentence: "Entities found in violation of Section 92(6) may face

In [9]:
# Method to Evaluate the Model on a Given Dataset
def evaluate_model_on_dataset(model, dataset_path):
    # Load the dataset
    data = pd.read_csv(dataset_path)
    data=data.sample(frac=1, random_state=42)

    # Ensure labels are numeric
    data['label'] = data['label'].map({'obligation': 1, 'non-obligation': 0})

    texts = data['text'].to_list()
    labels = data['label'].to_list()

    preds = []
    model.eval()
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=max_len,
                padding='max_length',
                return_attention_mask=True,
                return_tensors='pt',
                truncation=True
            )
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).cpu().numpy()[0]
            preds.append(prediction)

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')



In [10]:
# Evaluate model performance on a sample dataset
evaluate_model_on_dataset(model, './test_obligation_data.csv')  # Change to your dataset path

Accuracy: 0.48356164383561645
Precision: 0.9151515151515152
Recall: 0.29377431906614787
F1 Score: 0.44477172312223867


In [11]:
evaluate_model_on_dataset(model, './extended_obligation_classification_dataset_V2.csv')  # Change to your dataset path

Accuracy: 0.5556447808237054
Precision: 0.5393859184753838
Recall: 0.9852440628142647
F1 Score: 0.6971222923137973
